# SQL

DROP TABLE covid_result;
DROP TABLE covid_live;
DROP TABLE covid_vaccine_center;

CREATE TABLE covid_result(
sequence int PRIMARY KEY,
result_date DATE,
location VARCHAR(20),
increment_count INT,
total_count INT
);

CREATE TABLE covid_live(
live_date date PRIMARY KEY,
seoul INT,
incheon INT,
gwangju INT,
daejeon INT,
daegu INT,
busan INT,
ulsan INT,
sejong INT,
gyeonggi INT,
gangwon INT,
chungbuk INT,
chungnam INT,
jeonbuk INT,
jeonnam INT,
gyeongbuk INT,
gyeongnam INT,
jeju INT
);

CREATE TABLE covid_vaccine_center(
center_name VARCHAR(200),
facility_name VARCHAR(200),
phone_number VARCHAR(40),
location VARCHAR(20),
address VARCHAR(200) PRIMARY KEY,
zip_code VARCHAR(20),
lat DOUBLE,
lng DOUBLE
);

SELECT * FROM covid_result;
SELECT * FROM covid_live;
SELECT * FROM covid_vaccine_center;

COMMIT;

In [2]:
# 공공데이터활용지원센터_보건복지부 코로나19 시·도발생 현황
# https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043378

import requests
from bs4 import BeautifulSoup
import mysql.connector
import datetime as dt

date = dt.datetime.now() # 20210514일 경우 20210513의 확진자 집계 정보가 나옴
date = date.strftime('%Y%m%d')
print(date)

url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson' # 서비스 URL
apiKey = 'serviceKey=' + 'API KEY' # 인증키
pageNo = 'pageNo=' + str(1) # 페이지 번호
numOfRows = 'numOfRows=' + str(200) # 한 페이지 결과 수
startCreateDt = 'startCreateDt=' + str(date) # 데이터 생성일 시작 범위
endCreateDt = 'endCreateDt=' + str(date) # 데이터 생성일 종료 범위
params = pageNo + '&' + numOfRows  + '&' + startCreateDt  + '&' + endCreateDt # 파라메터 합치는 과정
open_url = url + '?' + apiKey + '&' + params # 요청 URL 완성

print(open_url + '\n') # 요청한 링크, 링크 누르면 데이터 확인 가능

res = requests.get(open_url) # 응답받은 데이터를 res에 저장
soup = BeautifulSoup(res.content, 'html.parser') # bs에 알맞게 변환

items = soup.findAll('item') # 데이터에서 item 태그들을 찾아 ResultSet으로 반환

result = [] # item 데이터를 넣을 tuple 생성

for one in items:
    print('sequence : ' + one.find('seq').string)
    print('수정 시간 : ' + one.find('createdt').string) # 나중에는 수정 시간 하나만 출력해도 될 거 같음
    print('지역 : ' + one.find('gubun').string)
    print('일일 확진자 수 : ' + one.find('incdec').string)
    print('누적 확진자 수 : ' + one.find('defcnt').string)
    print('누적 완치자 수 : ' + one.find('isolclearcnt').string)
    print('누적 사망자 수 : ' + one.find('deathcnt').string)
    sequence = str(one.find('seq').string)
    createDt = one.find('createdt').string[:10] # ex) 2021-05-18
    splitDate = createDt.split('-')
    #실제로는 하루 전 날의 통계 데이터이기 때문에 str을 datetime로 변환 후 -1일 연산
    date = dt.date(int(splitDate[0]), int(splitDate[1]), int(splitDate[2])) - dt.timedelta(days=1)
    result_date = date.strftime('%Y-%m-%d') # 양식에 맞게 str로 변환
    location = str(one.find('gubun').string)
    increment_count = str(one.find('incdec').string)
    total_count = str(one.find('defcnt').string)
    clear_count = str(one.find('isolclearcnt').string)
    death_count = str(one.find('deathcnt').string)
    
    data = (sequence, result_date, location, increment_count, total_count, clear_count, death_count) # tuple 생성
    result.append(data) # list에 추가
    print()

# MariaDB 커넥션 생성

try:
    conn = mysql.connector.connect(
        user="DB USER",
        password="DB PASSWORD",
        host="localhost",
        port=3306,
        database="DB NAME"
    )
    print(conn)
except mysql.connector.Error as e:
    print(e)


cur = conn.cursor() # 커서 생성

sql = "INSERT IGNORE INTO covid_result VALUES(%s, %s, %s, %s, %s, %s, %s)" # 쿼리 생성

cur.executemany(sql, result) # result 만큼의 쿼리 실행


conn.commit() # 커밋

# close 작업 실행
cur.close() 
conn.close()

20210615
http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey=WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D&pageNo=1&numOfRows=200&startCreateDt=20210612&endCreateDt=20210615

sequence : 10153
수정 시간 : 2021-06-15 09:37:45.206
지역 : 검역
일일 확진자 수 : 9
누적 확진자 수 : 3964
누적 완치자 수 : 3783
누적 사망자 수 : 5

sequence : 10152
수정 시간 : 2021-06-15 09:37:45.206
지역 : 제주
일일 확진자 수 : 6
누적 확진자 수 : 1198
누적 완치자 수 : 1085
누적 사망자 수 : 1

sequence : 10151
수정 시간 : 2021-06-15 09:37:45.206
지역 : 경남
일일 확진자 수 : 3
누적 확진자 수 : 5049
누적 완치자 수 : 4765
누적 사망자 수 : 20

sequence : 10150
수정 시간 : 2021-06-15 09:37:45.206
지역 : 경북
일일 확진자 수 : 5
누적 확진자 수 : 4822
누적 완치자 수 : 4617
누적 사망자 수 : 86

sequence : 10149
수정 시간 : 2021-06-15 09:37:45.206
지역 : 전남
일일 확진자 수 : 3
누적 확진자 수 : 1556
누적 완치자 수 : 1433
누적 사망자 수 : 15

sequence : 10148
수정 시간 : 2021-06-15 09:37:45.206
지역 : 전북
일일 확진자 수 : 4
누적 확진자 수 : 2298
누적 완치자 수 : 2190
누적 사망자 수 : 58

sequence : 10147
수정 시간 : 2021-0

In [41]:
# 공공데이터활용지원센터_코로나19 예방접종센터 현황
# https://www.data.go.kr/tcs/dss/selectFileDataDetailView.do?publicDataPk=15077603
# XML 파일의 태그들이 불편하게 되어있고 태그 인식 오류가 발생하여 bs 대신 et를 사용함

import requests
import xml.etree.ElementTree as et
import mysql.connector

url = 'https://api.odcloud.kr/api/15077586/v1/centers' # 서비스 URL
apiKey = 'serviceKey=' + 'API KEY' # 인증키
page = 'page=' + str(1) # 페이지 번호
perPage = 'perPage=' + str(500) # 한 페이지 결과 수
returnType = 'returnType=' + 'XML' # JSON, XML 사용 가능, 대소문자 구분 해야 함
params = page + '&' + perPage + '&' + returnType # 파라메터 합치는 과정
open_url = url + '?' + apiKey + '&' + params # 요청 URL 완성

print(open_url + '\n') # 요청한 링크, 링크 누르면 데이터 확인 가능

res = requests.get(open_url) # 응답받은 데이터를 res에 저장
tree = et.fromstring(res.content) # et에 알맞게 변환

# col 태그 안의 내용들이 순서가 일정하지 않기 때문에 dictionary로 받아온 후 list에 재정렬
dictionaryList = [] # item 내부에 있는 dictionary를 넣을 list 생성

for item in tree.iter('item'): # 데이터에서 item 태그들을 찾고
    dic = {} # dictionary 생성
    for col in item.iter('col'): # item 태그안의 col 태그를 찾아 하나씩 반복
        dic[col.get('name')] = col.text # 찾은 정보를 dictionary에 key와 value를 알맞게 추가
        #print(col.get('name') ,":", col.text)
    #print(dic)
    dictionaryList.append(dic) # list에 추가
    #print()

result = [] # 쿼리를 편하게 사용하기 위해 위의 list 데이터를 정리하기 위한 list 생성
for item in dictionaryList:
    # list안의 dictionary 데이터를 쿼리에 알맞게 정렬
    data = (item['centerName'], item['facilityName'], item['phoneNumber'], item['sido'], item['address'], item['zipCode'], item['lat'], item['lng'])
    print(data)
    result.append(data) # list에 추가
    
# MariaDB 커넥션 생성

try:
    conn = mysql.connector.connect(
        user="DB USER",
        password="DB PASSWORD",
        host="localhost",
        port=3306,
        database="DB NAME"
    )
    print(conn)
except mysql.connector.Error as e:
    print(e)


cur = conn.cursor() # 커서 생성

sql = "INSERT IGNORE INTO covid_vaccine_center VALUES(%s, %s, %s, %s, %s, %s, %s, %s)" # 쿼리 생성

cur.executemany(sql, result) # 쿼리 실행

conn.commit() # 커밋 

# close 작업 실행
cur.close()
conn.close()

https://api.odcloud.kr/api/15077586/v1/centers?serviceKey=WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D&page=1&perPage=500&returnType=XML

('코로나19 중앙 예방접종센터', '국립중앙의료원 D동', '02-2260-7114', '서울특별시', '서울특별시 중구 을지로 39길 29', '04562', '37.567817', '127.004501')
('코로나19 중부권역 예방접종센터', '천안시 실내배드민턴장 1층', None, '충청남도', '충청남도 천안시 동남구 천안대로 357', '31212', '36.779887', '127.164717')
('코로나19 호남권역 예방접종센터', '조선대학교병원 의성관 5층', '062-220-3739', '광주광역시', '광주광역시 동구 필문대로 365', '61452', '35.139465', '126.925563')
('코로나19 영남권역 예방접종센터', '양산 부산대병원 기숙사동 1층', '055-360-6701', '경상남도', '경상남도 양산시 물금읍 금오로 20', '50612', '35.3239', '129.009337')
('코로나19 대구광역시 중구 예방접종센터', '계명대학교 대구동산병원 별관', '053-661-3955', '대구광역시', '대구시 중구 달성로 56', '41931', '35.869985', '128.583716')
('코로나19 서울특별시 성동구 예방접종센터', '성동구청 대강당(3층)', '02-2286-5084', '서울특별시', '서울특별시 성동구 고산자로 270', '04750', '37.563457', '127.036981')
('코로나19 부산광역시 부산진구 예방접종센터', '부산시민공원 시민사랑채', '051-605-8633', '부산광역시', '부산 부산진구 시민공

In [39]:
# 공공데이터활용지원센터_코로나19 예방접종 위탁의료기관 조회서비스
# https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15081240
# XML 파일의 태그들이 불편하게 되어있고 태그 인식 오류가 발생하여 bs 대신 et를 사용함

import requests
import xml.etree.ElementTree as et
import mysql.connector
import urllib
import json

def getGPS_coordinate_for_KAKAO(address):
    headers = {
        'Content-Type': 'application/json; charset=utf-8',
        'Authorization': 'KakaoAK {}'.format('21164a56a8040a1a452117838952abd8')
    }
    address = address.encode("utf-8")
    p = urllib.parse.urlencode({
            'query': address
    })
    result = requests.get("https://dapi.kakao.com/v2/local/search/address.json", headers=headers, params=p)
    return result.json()

url = 'https://api.odcloud.kr/api/apnmOrg/v1/list' # 서비스 URL
apiKey = 'serviceKey=' + 'API KEY' # 인증키
page = 'page=' + str(1) # 페이지 번호
perPage = 'perPage=' + str(20000) # 한 페이지 결과 수
returnType = 'returnType=' + 'XML' # JSON, XML 사용 가능, 대소문자 구분 해야 함
params = page + '&' + perPage + '&' + returnType # 파라메터 합치는 과정
open_url = url + '?' + apiKey + '&' + params # 요청 URL 완성

print(open_url + '\n') # 요청한 링크, 링크 누르면 데이터 확인 가능

res = requests.get(open_url) # 응답받은 데이터를 res에 저장
tree = et.fromstring(res.content) # et에 알맞게 변환

# col 태그 안의 내용들이 순서가 일정하지 않기 때문에 dictionary로 받아온 후 list에 재정렬
dictionaryList = [] # item 내부에 있는 dictionary를 넣을 list 생성

for item in tree.iter('item'): # 데이터에서 item 태그들을 찾고
    dic = {} # dictionary 생성
    for col in item.iter('col'): # item 태그안의 col 태그를 찾아 하나씩 반복
        dic[col.get('name')] = col.text # 찾은 정보를 dictionary에 key와 value를 알맞게 추가
        #print(col.get('name') ,":", col.text)
    #print(dic)
    dictionaryList.append(dic) # list에 추가
    #print()

result = [] # 쿼리를 편하게 사용하기 위해 위의 list 데이터를 정리하기 위한 list 생성
for item in dictionaryList:
    try:
        # list안의 dictionary 데이터를 쿼리에 알맞게 정렬
        center_name = '코로나19 ' + item['orgnm'] + ' 위탁의료기관'
        facility_name = item['orgnm']
        phone_number = item['orgTlno']
        location = item['orgZipaddr'].split(' ')[0]
        address = item['orgZipaddr']
        zip_code = '00000'
        search_result = getGPS_coordinate_for_KAKAO(address)
        lat = search_result['documents'][0]['road_address']['y']
        lng = search_result['documents'][0]['road_address']['x']
    except:
        print('continue')
        continue
    data = (center_name, facility_name, phone_number, location, address, zip_code, lat, lng)
    print(data)
    result.append(data) # list에 추가
    
# MariaDB 커넥션 생성

try:
    conn = mysql.connector.connect(
        user="DB USER",
        password="DB PASSWORD",
        host="localhost",
        port=3306,
        database="DB NAME"
    )
    print(conn)
except mysql.connector.Error as e:
    print(e)


cur = conn.cursor() # 커서 생성

sql = "INSERT IGNORE INTO covid_vaccine_center VALUES(%s, %s, %s, %s, %s, %s, %s, %s)" # 쿼리 생성

cur.executemany(sql, result) # 쿼리 실행

conn.commit() # 커밋 

# close 작업 실행
cur.close()
conn.close()


https://api.odcloud.kr/api/apnmOrg/v1/list?serviceKey=WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D&page=1&perPage=20000&returnType=XML

continue
('코로나19 서울대학교병원 위탁의료기관', '서울대학교병원', '02-2072-0534', '서울특별시', '서울특별시 종로구 대학로 101, (연건동)', '00000', '37.5804523853297', '126.997196437908')
('코로나19 서울적십자병원 위탁의료기관', '서울적십자병원', '02-2002-8871', '서울특별시', '서울특별시 종로구 새문안로 9, (평동) 적십자병원', '00000', '37.5667630123693', '126.967202493985')
('코로나19 학교법인 고려중앙학원 고려대학교의과대학부속병원(안암병원) 위탁의료기관', '학교법인 고려중앙학원 고려대학교의과대학부속병원(안암병원)', '02-2199-3770', '서울특별시', '서울특별시 성북구 고려대로 73, (안암동5가) 고려대병원', '00000', '37.5871109699535', '127.026357234716')
('코로나19 서울특별시 동부병원 위탁의료기관', '서울특별시 동부병원', '02-920-9115', '서울특별시', '서울특별시 동대문구 무학로 124, (용두동)', '00000', '37.5753539913466', '127.031512678311')
('코로나19 삼육서울병원 위탁의료기관', '삼육서울병원', '02-2210-3655', '서울특별시', '서울특별시 동대문구 망우로 82, (휘경동) 삼육서울병원', '00000', '37.5862119387672', '127.06355328121')
('코로나19 경희대학교병원 위탁의료기관', '경희대학교병원', '02-9

('코로나19 서울현대병원 위탁의료기관', '서울현대병원', '02-902-9119', '서울특별시', '서울특별시 강북구 도봉로 374, (번동, 서울현대병원)', '00000', '37.6402559204261', '127.028504374531')
('코로나19 영동병원 위탁의료기관', '영동병원', '02-469-0019', '서울특별시', '서울특별시 동대문구 한천로 4, (413-8)) (장안동,외1필지', '00000', '37.5632449294324', '127.058553733243')
('코로나19 신일병원 위탁의료기관', '신일병원', '02-903-5121', '서울특별시', '서울특별시 강북구 덕릉로 73, (수유동)', '00000', '37.6347745804936', '127.021825891177')
('코로나19 웰튼병원 위탁의료기관', '웰튼병원', '02-2690-2000', '서울특별시', '서울특별시 강서구 강서로 318, (내발산동)', '00000', '37.5540279731359', '126.837149511419')
('코로나19 포미즈여성병원 위탁의료기관', '포미즈여성병원', '02-2651-7500', '서울특별시', '서울특별시 양천구 공항대로 610, (목동, 정종일산부인과) 포미즈여성병원', '00000', '37.546789059764', '126.87235841377')
('코로나19 나누리병원 위탁의료기관', '나누리병원', '02-1688-9797', '서울특별시', '서울특별시 강남구 언주로 731, (논현동)', '00000', '37.5203066071262', '127.033979290041')
('코로나19 정동병원 위탁의료기관', '정동병원', '02-810-7900', '서울특별시', '서울특별시 동작구 양녕로 283, (상도1동)', '00000', '37.5049498152225', '126.949438704753')
('코로나19 의료법인제민의료재단동서병원 위탁의료기관', '

('코로나19 9988병원 위탁의료기관', '9988병원', '02-2297-9988', '서울특별시', '서울특별시 성동구 왕십리로 269, (행당동)', '00000', '37.5586057086894', '127.037479052118')
continue
('코로나19 성북우리아이들병원 위탁의료기관', '성북우리아이들병원', '02-912-0100', '서울특별시', '서울특별시 성북구 동소문로46길 10, (하월곡동) 성북우리아이들병원', '00000', '37.6068633235053', '127.029359244458')
continue
('코로나19 장튼위튼병원 위탁의료기관', '장튼위튼병원', '02-878-7119', '서울특별시', '서울특별시 관악구 남부순환로 1867, (봉천동)', '00000', '37.4798286942741', '126.957305110343')
('코로나19 강서바른세상병원 위탁의료기관', '강서바른세상병원', '02-2603-0300', '서울특별시', '서울특별시 강서구 금낭화로 9, (방화동) 바른세상병원빌딩 전체', '00000', '37.5661373795288', '126.811603359943')
('코로나19 세솟는정형외과병원 위탁의료기관', '세솟는정형외과병원', '02-1577-3631', '서울특별시', '서울특별시 마포구 백범로 26, (신수동) 세솟는병원', '00000', '37.5523725217444', '126.937096194761')
('코로나19 강북으뜸병원 위탁의료기관', '강북으뜸병원', '02-1522-1746', '서울특별시', '서울특별시 강북구 도봉로 187, (미아동) 지하1층, 2층~5층', '00000', '37.6254205559935', '127.026184524499')
('코로나19 서울본브릿지병원 위탁의료기관', '서울본브릿지병원', '02-1899-0090', '서울특별시', '서울특별시 강남구 도곡로 214, (도곡동) 롤스웰빌딩 지하2층, 지하1층,

('코로나19 박현태내과의원 위탁의료기관', '박현태내과의원', '02-578-9191', '서울특별시', '서울특별시 강남구 개포로 224, (개포동) 준빌딩', '00000', '37.4785685929442', '127.047295152937')
('코로나19 김인호소아과의원 위탁의료기관', '김인호소아과의원', '02-423-7359', '서울특별시', '서울특별시 송파구 송파대로 380, (송파동, 2층) 행우빌딩 2층 김인호소아청소년과', '00000', '37.5016748314922', '127.110846006168')
('코로나19 조내과의원 위탁의료기관', '조내과의원', '02-3426-6364', '서울특별시', '서울특별시 강동구 양재대로 1626, (명일동)', '00000', '37.5506474264789', '127.144288436536')
('코로나19 조승복의원 위탁의료기관', '조승복의원', '02-707-0022', '서울특별시', '서울특별시 용산구 원효로 155, (원효로2가)', '00000', '37.5350390914786', '126.960316170142')
('코로나19 한세외과의원 위탁의료기관', '한세외과의원', '02-939-3993', '서울특별시', '서울특별시 노원구 동일로 1624, (상계동) 통일상가', '00000', '37.6733538101899', '127.055850095666')
('코로나19 최재득내과의원 위탁의료기관', '최재득내과의원', '02-591-1672', '서울특별시', '서울특별시 서초구 방배로 177, (방배동)', '00000', '37.4892992684081', '126.99228929275')
continue
('코로나19 삼양제일내과의원 위탁의료기관', '삼양제일내과의원', '02-984-5533', '서울특별시', '서울특별시 강북구 솔매로 59, (미아동) 2층', '00000', '37.6249897572285', '127.018818425469')

('코로나19 송영석비뇨기과의원 위탁의료기관', '송영석비뇨기과의원', '02-416-3875', '서울특별시', '서울특별시 송파구 가락로 84, (석촌동) 3층', '00000', '37.5003859826032', '127.105690808919')
('코로나19 김화내과의원 위탁의료기관', '김화내과의원', '02-537-1717', '서울특별시', '서울특별시 서초구 신반포로 219, (잠원동, 반포쇼핑) 8동 302호', '00000', '37.5075121874388', '127.008065342049')
continue
('코로나19 황내과의원 위탁의료기관', '황내과의원', '02-885-5348', '서울특별시', '서울특별시 관악구 신림로11길 9, (신림동) 2층', '00000', '37.4700559715659', '126.93900718485')
('코로나19 정내과의원 위탁의료기관', '정내과의원', '02-532-4794', '서울특별시', '서울특별시 서초구 서초중앙로 230, (반포동, 반포프라자빌딩) 303호', '00000', '37.5021563648911', '127.012024373585')
('코로나19 양의원 위탁의료기관', '양의원', '02-413-0019', '서울특별시', '서울특별시 송파구 올림픽로4길 40, (잠실동, 우성아파트비상가) 3층 33호', '00000', '37.5071938064643', '127.074204083657')
('코로나19 한진우의원 위탁의료기관', '한진우의원', '02-418-1470', '서울특별시', '서울특별시 송파구 백제고분로 349, (석촌동) 가동 2층', '00000', '37.5045357717245', '127.104951127071')
('코로나19 신내과의원 위탁의료기관', '신내과의원', '02-977-1777', '서울특별시', '서울특별시 노원구 섬밭로 209, (하계동, 벽산상가) 206호', '00000', '37.634930666229', '

...

('코로나19 태전성모내과의원 위탁의료기관', '태전성모내과의원', '031-798-7975', '경기도', '경기도 광주시 태전동로 23, (태전동) 메디프라자 2층 201,202호', '00000', '37.3866584472733', '127.23397387169')
('코로나19 라파가정의학과의원 위탁의료기관', '라파가정의학과의원', '031-879-2500', '경기도', '경기도 의정부시 비우로 110, (녹양동) 2층', '00000', '37.7582615531801', '127.039194168303')
('코로나19 성모별소아청소년과의원 위탁의료기관', '성모별소아청소년과의원', '032-555-0504', '인천광역시', '인천광역시 계양구 장제로 910, (병방동) 1층 102호', '00000', '37.5476421013661', '126.741149616498')
('코로나19 굿모닝신경외과의원 위탁의료기관', '굿모닝신경외과의원', '031-855-0332', '경기도', '경기도 의정부시 체육로 306-28, (녹양동) 굿모닝프라자 2층', '00000', '37.7598979630194', '127.041348733231')
('코로나19 153연세연합의원 위탁의료기관', '153연세연합의원', '031-469-1006', '경기도', '경기도 안양시 만안구 안양로 142, (안양동) 3층', '00000', '37.3872321260631', '126.931544948649')
('코로나19 온소아청소년과의원 위탁의료기관', '온소아청소년과의원', '032-833-0484', '인천광역시', '인천광역시 연수구 해돋이로 165, (송도동) 차오름 프라자 403호', '00000', '37.3947414835124', '126.645088161114')
('코로나19 에코이비인후과의원 위탁의료기관', '에코이비인후과의원', '031-504-3838', '경기도', '경기도 시흥시 은계번영길 11, (은행동) 401호', '00

('코로나19 건강드림내과의원 위탁의료기관', '건강드림내과의원', '031-564-3838', '경기도', '경기도 남양주시 다산순환로 350, (다산동) 5층 508,509,510호', '00000', '37.6256261548853', '127.155370161769')
('코로나19 센트럴이비인후과의원 위탁의료기관', '센트럴이비인후과의원', '032-439-8582', '인천광역시', '인천광역시 남동구 논고개로 73, (논현동) 3층 301호', '00000', '37.3982608936325', '126.726554861881')
('코로나19 건유의원 위탁의료기관', '건유의원', '031-581-7525', '경기도', '경기도 가평군 가평읍 가화로 47, 건유의원', '00000', '37.8241086137536', '127.515507421026')
('코로나19 서울위(WE)편한내과의원 위탁의료기관', '서울위(WE)편한내과의원', '031-264-5555', '경기도', '경기도 용인시 수지구 성복2로 92, (성복동) 402호', '00000', '37.315520709786', '127.075963758378')
('코로나19 이샘별소아청소년과의원 위탁의료기관', '이샘별소아청소년과의원', '032-432-2800', '인천광역시', '인천광역시 남동구 논고개로 77, (논현동) 에코타워 203호', '00000', '37.398437607938', '126.726253921698')
('코로나19 365힐링의원 위탁의료기관', '365힐링의원', '031-8023-5365', '경기도', '경기도 용인시 기흥구 죽전로 49, (보정동) 씨티프라자 301호', '00000', '37.3207320482137', '127.114263975259')
('코로나19 능곡하나내과의원 위탁의료기관', '능곡하나내과의원', '031-364-8393', '경기도', '경기도 시흥시 장현능곡로 188, (능곡동) SG프라자 3층 301호', '0

('코로나19 바른내과의원 위탁의료기관', '바른내과의원', '031-928-6767', '경기도', '경기도 의정부시 평화로 647, (의정부동) 미건메디컬프라자 204, 205호', '00000', '37.7497205265835', '127.044621857349')
('코로나19 드림이비인후과의원 위탁의료기관', '드림이비인후과의원', '031-985-9831', '경기도', '경기도 김포시 김포대로 709, (풍무동) 퍼스트블루 503~4호', '00000', '37.6122466597948', '126.731847690225')
('코로나19 양주삼성내과의원 위탁의료기관', '양주삼성내과의원', '031-928-5024', '경기도', '경기도 양주시 옥정로 214, (옥정동) 5층 502호', '00000', '37.8206905059303', '127.092118315849')
('코로나19 다함이비인후과의원 위탁의료기관', '다함이비인후과의원', '02-3158-5959', '경기도', '경기도 고양시 덕양구 향기4로 1, (향동동) 중흥메디프라자 301~302호', '00000', '37.6006924486084', '126.895196938768')
('코로나19 밀레니엄서울내과의원 위탁의료기관', '밀레니엄서울내과의원', '032-851-1001', '인천광역시', '인천광역시 연수구 컨벤시아대로 69, (송도동) 송도 밀레니엄 4층 402호', '00000', '37.3957153799876', '126.650955929786')
('코로나19 티엘씨의원 위탁의료기관', '티엘씨의원', '031-604-5005', '경기도', '경기도 성남시 분당구 정자일로 197, (정자동) 2층 202호', '00000', '37.3680299802465', '127.105520669444')
('코로나19 티엘씨소아청소년과의원 위탁의료기관', '티엘씨소아청소년과의원', '031-607-5005', '경기도', '경기도 성남시 분당구 정자일로 197

('코로나19 바로본365의원 위탁의료기관', '바로본365의원', '032-575-0365', '인천광역시', '인천광역시 서구 가정로 309, (석남동) 4층', '00000', '37.5124068163945', '126.672406368997')
('코로나19 삼성우리내과의원 위탁의료기관', '삼성우리내과의원', '02-2613-7588', '경기도', '경기도 광명시 철산로 32, (철산동) 4층', '00000', '37.4762473093475', '126.869796623164')
('코로나19 모두이비인후과의원 위탁의료기관', '모두이비인후과의원', '031-851-7585', '경기도', '경기도 의정부시 용민로 60, (용현동) 3층', '00000', '37.7365994268374', '127.085227306759')
('코로나19 365키즈맘의원 위탁의료기관', '365키즈맘의원', '031-365-3735', '경기도', '경기도 시흥시 승지로59번길 12, (능곡동) 시티타워 3층 302~303호', '00000', '37.3688929302135', '126.809033453285')
('코로나19 맑은코아이비인후과의원 위탁의료기관', '맑은코아이비인후과의원', '032-888-6159', '인천광역시', '인천광역시 미추홀구 낙섬중로 138, (용현동) 미추홀프라자 3층일부층', '00000', '37.4582782365127', '126.644709555722')
continue
('코로나19 다산민내과의원 위탁의료기관', '다산민내과의원', '031-523-2775', '경기도', '경기도 남양주시 다산중앙로123번길 22-8, (다산동) 리더스퀘어 305~308호', '00000', '37.6242893190323', '127.151031115002')
('코로나19 베스트내과의원 위탁의료기관', '베스트내과의원', '031-755-7542', '경기도', '경기도 성남시 수정구 수정로 174, (신흥동) 진영빌딩 301

continue
('코로나19 삼성드림소아청소년과의원 위탁의료기관', '삼성드림소아청소년과의원', '02-417-7722', '경기도', '경기도 하남시 신우실로 61, (감일동) 푸리마타워 4층 405~406호', '00000', '37.5099425024856', '127.161718965195')
('코로나19 청음이비인후과의원 위탁의료기관', '청음이비인후과의원', '02-488-9978', '경기도', '경기도 하남시 신우실로 61 푸리마타워 308~309호 (감일동)', '00000', '37.5099425024856', '127.161718965195')
('코로나19 친절한서울의원 위탁의료기관', '친절한서울의원', '031-886-7077', '경기도', '경기도 여주시 세종로 23, (홍문동) 명진빌딩 5층', '00000', '37.2957399508321', '127.636907811577')
continue
('코로나19 경기도화성시동탄보건소 위탁의료기관', '경기도화성시동탄보건소', '031-5189-4375', '경기도', '경기도 화성시 노작로 226-9, (석우동)', '00000', '37.2070640285148', '127.07860799095')
('코로나19 경기도화성시동부보건소 위탁의료기관', '경기도화성시동부보건소', '031-5189-1400', '경기도', '경기도 화성시 떡전골로 72-3, 2,3층 (병점동, 리치프라자) 2,3층', '00000', '37.2052954860397', '127.034885743223')
37.3091994369319 126.989370326635


In [ ]:
# 행정안전부_재난문자방송 발령현황
# https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=3058822
# 실시간 확진자 수 파악에 사용 예정

import requests
from bs4 import BeautifulSoup
import mysql.connector
import datetime as dt
import re
import time as t

# 프로그램 시작 후 현재 시간을 갱신 시간으로 입력, 이후에 데이터 갱신 시간을 비교하여 원하는 데이터 입력
update_date = dt.datetime.now()
# update_date = dt.datetime(2021, 5, 18, 11, 0, 0) # 임의의 갱신 시간
print('Program Start! Time : ' + update_date.strftime('%Y-%m-%d %H:%M:%S'))
update_date = update_date - dt.timedelta(minutes=30) # 프로그램 시작 30분 전 데이터부터 집계 시작

# API에서 사용되는 지역명과 DB에서 사용되는 지역명을 일치 시키기 위한 list
location_list = [
    '서울특별시', '인천광역시', '광주광역시', '대전광역시', '대구광역시', '부산광역시', '울산광역시', '세종특별자치시', '경기도', '강원도',
    '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도'
]
db_location_list = [
    'seoul', 'incheon', 'gwangju', 'daejeon', 'daegu', 'busan', 'ulsan', 'sejong', 'gyeonggi', 'gangwon',
    'chungbuk', 'chungnam', 'jeonbuk', 'jeonnam', 'gyeongbuk', 'gyeongnam', 'jeju'
]

url = 'http://apis.data.go.kr/1741000/DisasterMsg3/getDisasterMsg1List' # 서비스 URL
apiKey = 'serviceKey=' + 'API KEY' # 인증키
pageNo = 'pageNo=' + str(1) # 페이지 번호
numOfRows = 'numOfRows=' + str(240) # 한 페이지 결과 수
returnType = 'type=' + 'xml' # json, xml 사용 가능, type는 python 기본 함수라서 returnType으로 변수명 변경
params = pageNo + '&' + numOfRows + '&' + returnType # 파라메터 합치는 과정
open_url = url + '?' + apiKey + '&' + params # 요청 URL 완성

print(open_url + '\n') # 요청한 링크, 링크 누르면 데이터 확인 가능

# 1분 마다 while 반복, update_date 이후의 데이터를 입력.
# 입력 된 데이터가 있을 시 update_date 수정,
# 아닐 경우 update_date를 유지하고 코드 실행 반복
while(True):
    print('Start!')
    res = requests.get(open_url) # 응답받은 데이터를 res에 저장
    soup = BeautifulSoup(res.content, 'html.parser') # bs에 알맞게 변환

    items = soup.findAll('row') # 데이터에서 row 태그들을 찾아 ResultSet으로 반환

    result = [] # row 태그 안의 내용을 알맞게 넣을 list 생성

    for one in items:
        # 재난 문자 입력 시간들을 추출하기 위한 코드
        split_create_date = str(one.find('create_date').string).split(' ') # ex) 2021/05/18 19:01:01
        date = split_create_date[0] # ex) 2021/05/18
        time = split_create_date[1] # ex) 19:01:01
        split_date = date.split('/') 
        split_time = time.split(':')
        year = int(split_date[0]) # 년
        month = int(split_date[1]) # 월
        day = int(split_date[2]) # 일
        hour = int(split_time[0]) # 시
        minute = int(split_time[1]) # 분
        second = int(split_time[2]) # 초
        datetime = dt.datetime(year, month, day, hour, minute, second) # 추출한 시간들로 datetime 생성

        split_location = str(one.find('location_name').string).split(' ') # ex) 경기도 ㅇㅇ시
        location = split_location[0] # ex) 경기도

        msg = str(one.find('msg').string) # 재난 문자 내용
        
        # 출력 태그들
        #print('문자 발송 일시 : ' + datetime.strftime('%Y-%m-%d %H:%M:%S'))
        #print('문자 발송 날짜 : ' + date)
        #print('문자 발송 시간 : ' + time)
        #print('도, 광역, 특별시 : ' + location)
        #print('메시지 내용 : ' + msg)   
        #print()
        
        # 0시 또는 00시 기준 데이터를 생략하기위한  코드
        infoIndex = msg.find('0시')
        if msg[infoIndex - 1:infoIndex] == ' ':
            print('continue')
            continue
        elif msg[infoIndex - 1:infoIndex] == '0':
            print('continue')
            continue
        
        # 재난 문자 내용 중 '확진자'라는 단어가 시작하는 위치와 '명'으로 끝나는 위치의 index 추출
        startIndex = msg.find('확진자')
        endIndex = msg.find('명')
        
        # 확진자 수를 저장할 int 데이터 초기화
        count = 0
        
        # 두 index가 제대로 추출 되었으면
        if startIndex != -1 and endIndex != -1 and startIndex < endIndex:
            numMsg = msg[startIndex : endIndex + 1] # ex) 확진자 18명
            #print(numMsg)
            num = re.findall("\d+", numMsg) # numMsg 문자열에서 숫자만 추출 
            if(len(num) == 1): # 추출한 값 중 앞의 값을 우선으로
                count = int(num[0]) # 확진자 수 입력
                #print(count)
        #print()    
        
        # 확진자 수가 0명 초과하고 갱신되지 않은 데이터라면 
        if count > 0 and datetime > update_date:
            # 위에서 추출한 location 정보를 DB에 맞는 location 값으로 변경
            location = db_location_list[location_list.index(location)]
            data = [count, date, location] # 필요한 데이터를 list에 저장
            #print(data)
            result.append(data) # list에 추가
    
    # 삽입 될 값들 출력
    print('Result : ')
    print(result)
    
    # MariaDB 커넥션 생성
    try:
        conn = mysql.connector.connect(
            user="DB USER",
            password="DB PASSWORD",
            host="localhost",
            port=3306,
            database="DB NAME"
        )
        #print(conn)
    except mysql.connector.Error as e:
        print(e)
    
    cur = conn.cursor() # 커서 생성
    now = dt.datetime.now() # 현재 시간 받아와서
    now_str = update_date.strftime('%Y-%m-%d') # 연월일 str 데이터로 변경
    
    # covid_live table에 현재 날짜 데이터가 있는지 확인
    sql = "SELECT * FROM covid_live WHERE live_date = '" + now_str + "'" # 쿼리 생성
    cur.execute(sql) # 쿼리 실행
    sql_result = cur.fetchone() # 결과 값 입력

    # 결과값이 없다면 = 현재 날짜 데이터가 없음
    if sql_result is None: 
        print('Insert New Date! Time : ' + now_str)
        # 현재 날짜 데이터를 입력 할 쿼리 생성
        sql = "INSERT INTO covid_live VALUES ( '"+ now_str +"', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)" 
        cur.execute(sql) # 쿼리 실행
    
    for one in result:
        # 결과값을 입력할 쿼리 생성
        sql = "UPDATE covid_live SET " + one[2] + " = " + one[2] + " + %s WHERE live_date = %s"
        del one[2] # list에서 필요 없어진 데이터를 삭제
        cur.execute(sql, one) # 쿼리 실행
    
    conn.commit() # 커밋
    
    # close 문구 실행
    cur.close()
    conn.close()
    
    # 결과값이 있다면 = 데이터가 입력 되었으면
    if result:
        print('Updated Datetime!')
        update_date = dt.datetime.now() # 최근 데이터 갱신 시간을 현재 시간으로 변경
    else:
        now = dt.datetime.now() # 코드 실행 시간 출력
        print('No Update! Time : ' + now.strftime('%Y-%m-%d %H:%M:%S'))
        
    print('Recent Update Time : ' + update_date.strftime('%Y-%m-%d %H:%M:%S'))
    print('Sleep 60 Sec...')
    print()
    t.sleep(60) # 60초 뒤 코드 재실행

Program Start! Time : 2021-06-15 17:05:46
http://apis.data.go.kr/1741000/DisasterMsg3/getDisasterMsg1List?serviceKey=WGgoFgcwZMx8OpNmi7sn3E5dbRpaVyJnqmfNPDUoa%2FgwNlm2GulC88F9twJ5r%2B%2BHZsFP7UUTTzD92scFyTcoFA%3D%3D&pageNo=1&numOfRows=240&type=xml

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[[2, '2021/06/15', 'chungbuk']]
Updated Datetime!
Recent Update Time : 2021-06-15 17:05:46
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continu

No Update! Time : 2021-06-15 17:23:58
Recent Update Time : 2021-06-15 17:21:57
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 17:24:59
Recent Update Time : 2021-06-15 17:21:57
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 17:26:00
Recent Update Time : 2021-06-15 17:21:57
Sleep 60 Sec...

Start!
continue
continue
continue
continue

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 17:44:14
Recent Update Time : 2021-06-15 17:21:57
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 17:45:14
Recent Update Time : 2021-06-15 17:21:57
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
conti

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[[2, '2021/06/15', 'chungnam'], [1, '2021/06/15', 'gyeonggi'], [2, '2021/06/15', 'gyeonggi']]
Updated Datetime!
Recent Update Time : 2021-06-15 18:03:31
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 18:04:32
Recent Update Time : 2021-06-15 18:03:31
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
contin

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 18:25:58
Recent Update Time : 2021-06-15 18:07:45
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 18:26:58
Recent Update Time : 2021-06-15 18:07:45
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
conti

...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 20:54:19
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-06-15 21:17:35
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 21:34:09
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 20

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 23:04:37
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-15 23:05:37
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
conti

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-16 00:00:42
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-16 00:01:43
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
conti

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-16 01:11:16
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
Result : 
[]
No Update! Time : 2021-06-16 01:12:16
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
continue
conti

Start!
Result : 
[]
No Update! Time : 2021-06-16 09:12:53
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-06-16 09:13:56
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-06-16 09:14:59
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-06-16 09:16:02
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-06-16 09:17:06
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-06-16 09:18:09
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-06-16 09:19:12
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-06-16 09:20:15
Recent Update Time : 2021-06-15 18:27:59
Sleep 60 Sec...

Start!
Result : 
[]
No Update! Time : 2021-06-16 09:21:18
Recent Update 